In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from ll_xy import lonlat_to_xy
import os
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

In [2]:
wind_files = sorted(os.listdir('MET'))

col_names = ['station_num','year','month','day','hour','latitude',
             'longitude','temp','rel_humidity','SLP','wind_dir_deg','wind_spd_m_s',
             'Total_Cloud','Low_Cloud','Surf_Temp']

dfs = {}

for file in wind_files:
    
    station = int(file.split('_')[1][:2])
    
    
    df = pd.read_csv('MET/'+file,delim_whitespace=True,names=col_names)
    
    df.replace(to_replace=9999,value=np.nan,inplace=True)
    
    df = df[['year','month','day','hour','latitude','longitude','wind_dir_deg','wind_spd_m_s']]
        
    df.dropna(inplace=True)
    
    df['dt'] = [datetime.datetime(1900+y,m,d,h,0,0) for y,m,d,h in zip(df['year'],df['month'],df['day'],df['hour'])]
    df['delta_t_hrs']=[np.nan]+[(x-y).seconds/(60*60) for x,y in zip(df['dt'][1:],df['dt'][:-1])]
    
    df.dropna(inplace=True)
    
    df['year']+=1900
    
    dfs[station]=df

In [3]:
for station, df in dfs.items():
    
    df['x'],df['y']=lonlat_to_xy(np.array(df['longitude']),
                             np.array(df['latitude']),
                             hemisphere='n')
    df['dx']=[np.nan]+[(x1-x0) for x1,x0 in zip(df['x'][1:],df['x'][:-1])]
    df['dy']=[np.nan]+[(y1-y0) for y1,y0 in zip(df['y'][1:],df['y'][:-1])]
    df['dist'] = [np.sqrt(x**2+y**2) for x,y in zip(df['dx'],df['dy'])]
    df['station_speed_m_s'] = df['dist']/(df['delta_t_hrs']*(60*60))

    
        
    df['dlon']=[np.nan]+[(x-y) for x,y in zip(df['longitude'][1:],df['longitude'][:-1])]
    df['dlat']=[np.nan]+[(x-y) for x,y in zip(df['latitude'][1:],df['latitude'][:-1])]
    
    q=[geodesic.inv(long1, lat1, long2, lat2) for long1,lat1,long2,lat2 in zip(df['longitude'][:-1],
                                                                               df['latitude'][:-1],
                                                                               df['longitude'][1:],
                                                                               df['latitude'][1:],
                                                                              )]
    
    df['station_dir_deg']=[np.nan]+[x[1] for x in q]
    df['station_dir_deg'] %= 360 
    df['dist2']=[np.nan]+[x[2] for x in q]
    
    df['dir_stat_wind']=df['station_dir_deg']-df['wind_dir_deg']
    df['dir_stat_wind']=[(a+180)%360-180 for a in df['dir_stat_wind']]
    
    
    df['station']=station
    
    df.to_csv(f'tables/{station}.csv')
    
    
    
